In [33]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from time import gmtime, strftime

from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import sagemaker as sage

In [34]:
role = get_execution_role() # IAM Role
sess = sage.Session()

In [35]:
WORK_DATA = '../context/input/data/training'
prefix = 'cntkdemo-data' # S3 prefix (the folder that will be automagically created on the associated S3 space)
data_location = sess.upload_data(WORK_DATA,key_prefix=prefix)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/cntkdemo-ecr-4'.format(account, region)

In [ ]:
estimator = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

estimator.set_hyperparameters(input_dim=2,
                              num_classes=2, 
                              num_hidden_layers=2, 
                              hidden_layers_dim=50,
                              epochs=200,
                              minibatch_size=25,
                              init='normal',
                              verbose=2)

In [37]:
%%time
estimator.fit(data_location)

INFO:sagemaker:Creating training-job with name: cntkdemo-ecr-4-2018-01-21-13-26-49-604


...........................................................................
Using CNTK backend
Selected CPU as the process wide default device.
/root/anaconda3/envs/cntk-py27/lib/python2.7/site-packages/keras/backend/cntk_backend.py:19: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '
Starting the training.
Train on 20000 samples, validate on 1024 samples
Epoch 1/200
0s - loss: 0.7875 - acc: 0.5016 - val_loss: 0.6790 - val_acc: 0.5361
Epoch 2/200
0s - loss: 0.6740 - acc: 0.5785 - val_loss: 0.6678 - val_acc: 0.6113
Epoch 3/200
0s - loss: 0.6672 - acc: 0.6433 - val_loss: 0.6622 - val_acc: 0.6777
Epoch 4/200
0s - loss: 0.6617 - acc: 0.6892 - val_loss: 0.6568 - val_acc: 0.7227
Epoch 5/200
0s - loss: 0.6563 - acc: 0.7391 - val_loss: 0.6514 - val_acc: 0.7490
Epoch 6/200
0s - loss: 0.6509 - acc: 0.7715 - val_loss: 0.6461 - val_acc: 0.7813
Epoch 7/200


0s - loss: 0.2946 - acc: 0.9209 - val_loss: 0.2941 - val_acc: 0.9199
Epoch 81/200
0s - loss: 0.2923 - acc: 0.9221 - val_loss: 0.2920 - val_acc: 0.9189
Epoch 82/200
0s - loss: 0.2900 - acc: 0.9223 - val_loss: 0.2899 - val_acc: 0.9180
Epoch 83/200
0s - loss: 0.2878 - acc: 0.9224 - val_loss: 0.2875 - val_acc: 0.9199
Epoch 84/200
0s - loss: 0.2857 - acc: 0.9233 - val_loss: 0.2855 - val_acc: 0.9199
Epoch 85/200
0s - loss: 0.2836 - acc: 0.9234 - val_loss: 0.2835 - val_acc: 0.9189
Epoch 86/200
0s - loss: 0.2816 - acc: 0.9232 - val_loss: 0.2816 - val_acc: 0.9189
Epoch 87/200
0s - loss: 0.2796 - acc: 0.9234 - val_loss: 0.2796 - val_acc: 0.9189
Epoch 88/200
0s - loss: 0.2776 - acc: 0.9240 - val_loss: 0.2778 - val_acc: 0.9189
Epoch 89/200
0s - loss: 0.2758 - acc: 0.9238 - val_loss: 0.2760 - val_acc: 0.9189
Epoch 90/200
0s - loss: 0.2739 - acc: 0.9242 - val_loss: 0.2741 - val_acc: 0.9209
Epoch 91/200
0s - loss: 0.2721 - acc: 0.9239 - val_loss: 0.2724 - val_acc: 0.9219
Epoch 92/200
0s - loss: 0.270

===== Job Complete =====
CPU times: user 464 ms, sys: 48 ms, total: 512 ms
Wall time: 7min 39s


In [39]:
estimator.model_data

's3://sagemaker-us-east-2-012197625151/output/cntkdemo-ecr-4-2018-01-21-13-26-49-604/output/model.tar.gz'